In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path

from IPython import display

import cv2

In [2]:
from os import environ
environ['TENSORBOARD_BINARY'] = r'C:\Users\johna\Documents\A_Documents\Programming\Python\.my_envs\datascience\Scripts\tensorboard.exe'

In [3]:
%load_ext tensorboard

In [4]:
class Paths:
	CURRENT_MODEL = 'v7'
	CWD = Path.cwd()
	
	HISTORY_DIR = CWD/'history'
	MODEL_DIR = HISTORY_DIR/CURRENT_MODEL
	
	LOG_DIR = MODEL_DIR/'logs'
	SAVED_MODELS_DIR = MODEL_DIR/'saved_models'
	GENERATED_IMAGES_DIR = MODEL_DIR/'generated_images'
	
	CHECKPOINT_DIR = SAVED_MODELS_DIR/'checkpoints'
	SINGLE_IMAGE_DIR = GENERATED_IMAGES_DIR/'single'
	COLLECTION_DIR = GENERATED_IMAGES_DIR/'collection'
	
	DATASET_PATH = Path(r'e:\datasets\monet-paintings')

In [5]:
FOLDERS_TO_SETUP = [
	Paths.HISTORY_DIR,
	Paths.MODEL_DIR,
	
	Paths.SAVED_MODELS_DIR,
	Paths.GENERATED_IMAGES_DIR,
	
	Paths.CHECKPOINT_DIR,
	Paths.SINGLE_IMAGE_DIR,
	Paths.COLLECTION_DIR
]

In [6]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [7]:
def mkdir(path):
	path.mkdir(exist_ok=True)

In [8]:
def setup_folders(paths: list):
	for path in paths:
		mkdir(path)

In [9]:
def display_image(image):
	plt.imshow(image.astype('uint8'))

### data preparation

In [10]:
IMAGE_SHAPE = (64, 64, 3)
BATCHSIZE = 32

In [11]:
def normalize_images(images):
	return (images-127.5)/127.5

In [12]:
def denormalize_images(images):
	return np.array((images*127.5+127.5), dtype='uint8')

In [13]:
def resize_images(images, new_size=IMAGE_SHAPE[:-1], interpolation=cv2.INTER_AREA):
	return np.array([cv2.resize(image, new_size, interpolation=interpolation) for image in images], dtype='float32')

In [14]:
def prepare_images():
	images = np.array([plt.imread(image) for image in (Paths.DATASET_PATH/'monet_jpg').glob('*')], dtype='float32')
	assert images.shape == (images.shape[0], 256, 256, 3), f'input images have the wrong dimensions! {images.shape}'
	resized_images = resize_images(images)
	assert resized_images.shape == (images.shape[0], *IMAGE_SHAPE), f'input images have the wrong dimensions! {resized_images.shape}'
	normalized_images = normalize_images(resized_images)
	return normalized_images

In [15]:
images = prepare_images()
train_dataset = tf.data.Dataset.from_tensor_slices(images).shuffle(images.shape[0]).batch(BATCHSIZE).cache().prefetch(tf.data.AUTOTUNE)

### model building

In [16]:
FINAL_STEP = 1000*len(images)

In [17]:
# # for epochs instead of steps
# def custom_lr_schedule(current_epoch, final_epoch, init_lr, final_lr):
# 	if current_epoch >= final_epoch:
# 		return final_lr
# 	else:
# 		lr = -(np.sqrt(current_epoch * final_epoch) / final_epoch) * (init_lr - final_lr) + init_lr
# 		first_two_digits = int(np.abs(np.floor(np.log10(lr))) + 1)
# 		return round(lr, first_two_digits)

# from functools import partial
# generator_schedule = partial(custom_lr_schedule, final_epoch=1000, init_lr=1e-4, final_lr=1e-5)
# critic_schedule = partial(custom_lr_schedule, final_epoch=1000, init_lr=1e-4, final_lr=1e-7)

#### blocks

In [18]:
from tensorflow.keras import layers

In [19]:
def conv_block(x, n_filters, kernel_size=(3, 3), strides=(2, 2), padding='same'):
	x = layers.Conv2D(n_filters, kernel_size, strides, padding)(x)
	x = layers.LeakyReLU(0.2)(x)
	x = layers.Dropout(0.3)(x)
	return x

In [20]:
def normalized_conv_block(x, n_filters, kernel_size=(3, 3), strides=(2, 2), padding='same'):
	x = layers.Conv2D(n_filters, kernel_size, strides, padding)(x)
	x = layers.LeakyReLU(0.2)(x)
	x = layers.LayerNormalization()(x)
	x = layers.Dropout(0.3)(x)
	return x

In [21]:
def transposed_conv_block(x, n_filters, kernel_size=(3, 3), strides=(2, 2), padding=('same')):
	x = layers.Conv2DTranspose(n_filters, kernel_size, strides, padding, use_bias=False)(x)
	x = layers.LeakyReLU(0.2)(x)
	x = layers.BatchNormalization()(x)
	return x

In [22]:
def get_multiplied_image_dimension(current_dimensions, factor):
	return [dimension*factor for dimension in current_dimensions[-2:-4:-1]]

In [23]:
def resizing_conv_block(x, n_filters, kernel_size=(3, 3), strides=(1, 1), padding='same'):
	x = layers.UpSampling2D()(x)
	x = layers.Conv2D(n_filters, kernel_size, strides, padding)
	x = layers.LeakyReLU(0.2)(x)
	x = layers.LayerNormalization()(x)
	return x

In [24]:
def resizing_transposed_conv_block(x, n_filters, kernel_size=(3, 3), strides=(1, 1), padding='same'):
	x = layers.UpSampling2D()(x)
	x = layers.Conv2DTranspose(n_filters, kernel_size, strides, padding)(x)
	x = layers.LeakyReLU(0.2)(x)
	x = layers.BatchNormalization()(x)
	return x

#### models

In [27]:
# def wasserstein_loss(y_true, y_pred):
# 	return tf.keras.backend.mean(y_true*y_pred)

In [28]:
class Generator():
	def __init__(self, input_size, optimizer):
		self.input_size = input_size
		self.optimizer = optimizer
		self.model = self.build()
		
	def __call__(self, inputs, training=False):
		return self.model(inputs, training)
	
	def build(self):
		initial_dimensions = (8, 8)
		inputs = layers.Input(self.input_size)

		x = layers.Dense(np.product(initial_dimensions)*256, use_bias=False)(inputs)
		x = layers.LeakyReLU(0.2)(x)
		x = layers.BatchNormalization()(x)

		x = layers.Reshape((*initial_dimensions, 256))(x)
		
		x = resizing_transposed_conv_block(x, 256)
		x = resizing_transposed_conv_block(x, 256)

		# outputs = resizing_transposed_conv_block(x, 3)
		
		x = layers.Resizing(*get_multiplied_image_dimension(x.shape, 2), interpolation='nearest')(x)
		x = layers.Conv2DTranspose(3, kernel_size=(3, 3), padding='same')(x)
		outputs = tf.keras.activations.tanh(x)
		
		assert outputs.get_shape()[1:] == IMAGE_SHAPE, f'output tensor\'s shapes are wrong! {outputs.get_shape()}'

		model = tf.keras.Model(inputs, outputs, name='generator')

		return model
	
	def compute_loss(self, fake_predictions):
		return -tf.reduce_mean(fake_predictions)

In [29]:
class Critic():
	def __init__(self, input_size, optimizer):
		self.input_size = input_size
		self.optimizer = optimizer
		self.model = self.build()
		
	def __call__(self, inputs, training=False):
		return self.model(inputs, training=training)
	
	def build(self):
		inputs = layers.Input(self.input_size)

		x = normalized_conv_block(inputs, 64, (5, 5))
		x = normalized_conv_block(x, 128)

		x = layers.Flatten()(x)
		x = layers.Dropout(0.2)(x)
		outputs = layers.Dense(1)(x)

		model = tf.keras.Model(inputs, outputs, name='critic')

		return model
	
	def compute_loss(self, real_predictions, fake_predictions):
		real_loss = tf.reduce_mean(real_predictions)
		fake_loss = tf.reduce_mean(fake_predictions)
		return fake_loss - real_loss

In [30]:
class WGAN(tf.keras.Model):
	def __init__(self, generator, critic, latent_dim, batchsize=BATCHSIZE, critic_extra_steps=5, alpha=1e-4, beta1=0, beta2=0.9, lambd=10):
		super().__init__()
		self.generator = generator
		self.critic = critic
		self.optimizer = tf.keras.optimizers.Adam(alpha, beta1, beta2)
		self.latent_dim = latent_dim
		self.critic_extra_steps = critic_extra_steps
		self.batchsize = BATCHSIZE
		self.lambd = lambd
		self.last_epoch = 0

	def compute_gradient_penalty(self, generated_images, real_images):
		epsilon = tf.random.uniform((self.batchsize, 1, 1, 1), 0, 1)
		interpolated_generated_images = epsilon * real_images + (1-epsilon) * generated_images

		with tf.GradientTape() as gp_tape:
			gp_tape.watch(interpolated_generated_images)
			prediction = critic(interpolated_generated_images, training=True)

		grads = gp_tape.gradient(prediction, interpolated_generated_images)
		
		norms = tf.sqrt(tf.reduce_sum(tf.square(grads), axis=(1, 2, 3)))
		gradient_penalty = self.lambd * tf.reduce_mean(tf.square(norms - 1))

		return gradient_penalty

	def train_step(self, images):
		self.batchsize = tf.shape(images)[0]
		for _ in range(self.critic_extra_steps):
			noise = tf.random.normal(shape=(self.batchsize, self.latent_dim))
			
			with tf.GradientTape() as c_tape:
				generated_images = self.generator(noise, training=True)
				fake_predictions = self.critic(generated_images, training=True)
				real_predictions = self.critic(images, training=True)
			
				c_cost = self.critic.compute_loss(real_predictions, fake_predictions)
				gradient_penalty = self.compute_gradient_penalty(generated_images, images)
				c_loss = c_cost + gradient_penalty
		
			c_gradient = c_tape.gradient(c_loss, self.critic.model.trainable_variables)
			self.critic.optimizer.apply_gradients(zip(c_gradient, self.critic.model.trainable_variables))

		noise = tf.random.normal((self.batchsize, self.latent_dim))
		
		with tf.GradientTape() as g_tape:
			generated_images = self.generator(noise, training=True)
			fake_predictions = self.critic(generated_images, training=True)
			g_loss = self.generator.compute_loss(fake_predictions)
		
		g_gradient = g_tape.gradient(g_loss, self.generator.model.trainable_variables)
		self.generator.optimizer.apply_gradients(zip(g_gradient, self.generator.model.trainable_variables))
		
		return {'c_loss': c_loss, 'g_loss': g_loss}

#### callbacks

In [31]:
class GANMonitor(tf.keras.callbacks.Callback):
	def __init__(self, latent_dim=100):
		self.latent_dim = latent_dim
		
	def on_epoch_end(self, epoch, logs=None):
		noise = tf.random.normal((12, self.latent_dim))
		generated_images = self.model.generator(noise)
		generated_images = denormalize_images(generated_images)
	
		display.clear_output(True)
		
		f, axs = plt.subplots(3, 4, figsize=(20, 15))
		for i, ax in enumerate(axs.flatten()):
			ax.imshow(generated_images[i])
			ax.axis('off')
		plt.show()
		
		if epoch % 25 == 0:
			checkpoint.save(Paths.CHECKPOINT_DIR/'checkpoint')
			
		if epoch % 5 == 0:
			f.savefig(Paths.COLLECTION_DIR/f'image_at_epoch_{epoch}.png', format='png', dpi=100)

In [32]:
class CustomLearningRateSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
	def __init__(self, init_lr, final_lr, final_step):
		self.init_lr = tf.cast(init_lr, 'float32')
		self.final_lr = tf.cast(final_lr, 'float32')
		self.final_step = tf.cast(final_step, 'float32')
		
	def __call__(self, step):
		return tf.cond(
			tf.greater_equal(step, FINAL_STEP),
			true_fn=lambda: self.final_lr,
			false_fn=lambda: -(tf.sqrt(step * self.final_step) / self.final_step) * (self.init_lr - self.final_lr) + self.init_lr
		)

### training

In [33]:
LATENT_DIM = 100

In [34]:
gan_monitor = GANMonitor()

In [35]:
generator_schedule = CustomLearningRateSchedule(init_lr=1e-4, final_lr=1e-5, final_step=FINAL_STEP)
critic_schedule = CustomLearningRateSchedule(init_lr=1e-4, final_lr=1e-7, final_step=FINAL_STEP*5)

generator_optimizer = tf.keras.optimizers.Adam(generator_schedule, 0, 0.9)
critic_optimizer = tf.keras.optimizers.Adam(critic_schedule, 0, 0.9)

In [36]:
generator = Generator(LATENT_DIM, generator_optimizer)
critic = Critic(IMAGE_SHAPE, critic_optimizer)
checkpoint = tf.train.Checkpoint(
	generator_optimizer=generator.optimizer,
	critic_optimizer=critic.optimizer,
	generator=generator.model,
	critic=critic.model
)

In [37]:
tensorboard_cb = tf.keras.callbacks.TensorBoard(log_dir=Paths.LOG_DIR, histogram_freq=1)

In [38]:
wgan = WGAN(generator, critic, LATENT_DIM)
wgan.compile()

In [39]:
EPOCHS = 5
setup_folders(FOLDERS_TO_SETUP)
history = wgan.fit(train_dataset, shuffle=True, epochs=EPOCHS, callbacks=[gan_monitor, tensorboard_cb], verbose=1)
# history = wgan.fit(train_dataset, shuffle=True, epochs=EPOCHS, verbose=1)

10/10 [==============================] - 4s 437ms/step - c_loss: -5.7652 - g_loss: -12.6771


In [ ]:
%tensorboard --logdir Paths.LOG_DIR

In [ ]:
def save_models():
	wgan.generator.model.save(Paths.SAVED_MODELS_DIR/'generator')
	wgan.critic.model.save(Paths.SAVED_MODELS_DIR/'critic')
# save_models()

In [ ]:
f, axs = plt.subplots(1, 2, figsize=(30, 9))
axs[0].plot(wgan.history.history['c_loss'])
axs[0].title.set_text('critic loss')
axs[1].plot(wgan.history.history['g_loss'])
axs[1].title.set_text('generator loss')
plt.show()

In [ ]:
noise = tf.random.normal((16, 100))
generated_images = wgan.generator(noise)
generated_images = denormalize_images(generated_images)

display.clear_output(True)

for i in range(16):
	plt.subplot(4, 4, i+1)
	plt.imshow(generated_images[i])
	plt.axis('off')

plt.show()

TODO:
- redesign model saving architecture, redefine in classes á la official keras documenation, see if that actually brings any benefits
- prepare model for new dataset since the current monet paintings DS is quite small
- analyse performance, improve accordingly
- read more about performance, maybe redesign model architecture to a more efficient format

#### refrences
http://modelai.gettysburg.edu/2020/wgan/Resources/Lesson5/WGAN-GP.pdf  
https://keras.io/examples/generative/wgan_gp/  
https://github.com/caogang/wgan-gp/blob/master/gan_mnist.py  
https://developers.google.com/machine-learning/gan/loss  
https://www.youtube.com/watch?v=pG0QZ7OddX4  
https://distill.pub/2016/deconv-checkerboard/  